# [MXBoard — 助力 MXNet 数据可视化](https://zh.mxnet.io/blog/mxboard)

GitHub:[Logging MXNet Data for Visualization in TensorBoard](https://github.com/awslabs/mxboard#installation)

# MXBoard 快速上手指南

所有的记录 API 都定义在一个叫 `SummaryWriter` 的类当中，这个类含有诸如记录的文件地址、写文件的频率、写文件的队列大小等等信息，用户可以根据需求设置。当需要把当前数据记录成 TensorBoard 中某种数据类型时，用户只要调用相应的 API 即可。

画一个正态分布标准差逐渐缩小的数据分布图。首先定义一个写记录的对象如下，它会把数据写入到当前文件夹下的名为 `logs` 的文件夹。

In [1]:
import mxnet as mx
from mxboard import SummaryWriter

sw = SummaryWriter(logdir='D:/logs')

接着在每个循环里，用 MXNet 的随机正态分布算子创建一个 NDArray，把这个 NDArray 传给写数据的 API `add_histogram`，指定画分布图时 `bin` 的数量和当前的循环数。最后，和 Python 里常用的文件写入器一样，记得关闭这个 `SummaryWriter`。

In [2]:
for i in range(10):
    # create a normal distribution with fixed mean and decreasing std
    data = mx.nd.random.normal(loc=0, scale=10.0/(i+1), shape=(10, 3, 8, 8))
    sw.add_histogram(tag='norml_dist', values=data, bins=200, global_step=i)
sw.close()

为了看到效果图，打开命令行窗口，进入到当前文件夹，键入如下命令以打开 TensorBoard：
```sh
tensorboard --logdir=./logs --host=127.0.0.1 --port=8888
```

接着在浏览器地址栏输入 `127.0.0.1:8888`，点击 `HISTOGRAM`，就可以看到效果图了。

# 实战 MXBoard
- 监督模型训练
- 理解卷积神经网络的工作原理

## [训练 MNIST 模型](https://github.com/awslabs/mxboard/tree/master/examples/mnist)
借用 Gluon 里训练 MNIST 模型的 Python 程序，用 MXBoard 记录下交叉熵、训练和测试精度、参数的梯度数据分布，可以实时反映出模型训练的进度。

- 首先定义一个 `SummaryWriter` 的对象：

In [3]:
sw = SummaryWriter(logdir='D:/logs', flush_secs=5)

这里加了 `flush_secs=5` 是为了每五秒就写一次记录到文件，以便在浏览器中及时看到结果。接着在每个 `mini-batch` 循环结束时记录下交叉熵

```py
sw.add_scalar(tag='cross_entropy', value=L.mean().asscalar(), global_step=global_step)
```

In [24]:
from mxnet.gluon import nn
from mxnet import gluon, autograd
import numpy as np

net = nn.HybridSequential()
with net.name_scope():
    net.add(nn.Dense(128, activation='relu'))
    net.add(nn.Dense(64, activation='relu'))
    net.add(nn.Dense(10))


# data

def transformer(data, label):
    data = data.reshape((-1,)).astype(np.float32) / 255
    return data, label

batch_size = 64
train_data = gluon.data.DataLoader(
    gluon.data.vision.MNIST('E:/Data/MXNet/mnist',
                            train=True, transform=transformer),
    batch_size=batch_size, shuffle=True, last_batch='discard')

val_data = gluon.data.DataLoader(
    gluon.data.vision.MNIST('E:/Data/MXNet/mnist',
                            train=False, transform=transformer),
    batch_size=batch_size, shuffle=False)


def test(ctx):
    metric = mx.metric.Accuracy()
    for data, label in val_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        metric.update([label], [output])
    return metric.get()


ctx = mx.gpu(0)
net.initialize(ctx=ctx, init=mx.init.MSRAPrelu())
net.hybridize()
loss = gluon.loss.SoftmaxCrossEntropyLoss()
#trainer = gluon.Trainer(net.collect_params(), 'adadelta', {'rho': 0.9999})
trainer = gluon.Trainer(net.collect_params(), 'rmsprop', {'learning_rate': 0.01, 'gamma1': 0.9})

In [25]:
params = net.collect_params()
param_names = list(params.keys())

In [39]:
# define a summary writer that logs data and flushes to the file every 5 seconds
sw = SummaryWriter(logdir='D:/logs', flush_secs=5)
global_step = 0
ctx = mx.gpu(0)
metric = mx.metric.Accuracy()
epoch = 0
metric.reset()
for i, (data, label) in enumerate(train_data):
    # Copy data to ctx if necessary
    data = data.as_in_context(ctx)
    label = label.as_in_context(ctx)
    # Start recording computation graph with record() section.
    # Recorded graphs can then be differentiated with backward.
    with autograd.record():
        output = net(data)
        L = loss(output, label)
        sw.add_scalar('cross_entropy', {'mnist' : L.mean().asscalar()}, global_step=global_step)
        global_step += 1
    L.backward()
    # take a gradient step with batch_size equal to data.shape[0]
    trainer.step(data.shape[0])
    # update metric at last.
    metric.update([label], [output])
    if i % 100 == 0 and i > 0:
        # how many batches to wait before logging training status
        name, train_acc = metric.get()
        print(('[Epoch %d Batch %d] Training: %s=%f' %(epoch, i, name, train_acc)))
    if i == 0:
        sw.add_image('mnist_first_minibatch', data.reshape((batch_size, 1, 28, 28)), epoch)
if epoch == 0:
    sw.add_graph(net)

[Epoch 0 Batch 100] Training: accuracy=0.961634
[Epoch 0 Batch 200] Training: accuracy=0.961210
[Epoch 0 Batch 300] Training: accuracy=0.960029
[Epoch 0 Batch 400] Training: accuracy=0.960061
[Epoch 0 Batch 500] Training: accuracy=0.960454
[Epoch 0 Batch 600] Training: accuracy=0.960301
[Epoch 0 Batch 700] Training: accuracy=0.961216
[Epoch 0 Batch 800] Training: accuracy=0.961513
[Epoch 0 Batch 900] Training: accuracy=0.961362


In [42]:
grads = [i.grad() for i in net.collect_params().values()]
assert len(grads) == len(param_names)
# logging the gradients of parameters for checking convergence
for i, name in enumerate(param_names):
    sw.add_histogram(tag=name, values=grads[i], global_step=epoch, bins=1000)
name, acc = metric.get()
# logging training accuracy
sw.add_scalar(tag='train_acc', value=acc, global_step=epoch)

name, val_acc = test(ctx)
# logging the validation accuracy
sw.add_scalar(tag='valid_acc', value=val_acc, global_step=epoch)

在每个 `epoch` 结束时记录下参数的梯度为 `HISTOGRAM`，记录下训练和测试精度为 `SCALAR`，
```py
grads = [i.grad() for i in net.collect_params().values()]
assert len(grads) == len(param_names)
# logging the gradients of parameters for checking convergence
for i, name in enumerate(param_names):
    sw.add_histogram(tag=name, values=grads[i], global_step=epoch, bins=1000)

name, acc = metric.get()
# logging training accuracy
sw.add_scalar(tag='train_acc', value=acc, global_step=epoch)

name, val_acc = test(ctx)
# logging the validation accuracy
sw.add_scalar(tag='valid_acc', value=val_acc, global_step=epoch)
```

然后运行 Python 程序，并运行 TensorBoard，就可以在浏览器中看到以下效果了。小伙伴们可以尝试着用 MXBoard 监督训练更复杂神经网络。更多本实例的代码和解说请点击[这里](https://github.com/reminisce/mxboard-demo#monitoring-training-mnist-model)。

## 可视化卷积层的 filters 和 feature maps

将卷积层的 `filters` 和 `feature maps` 当成图片可视化有两个意义：
- 特征平滑规律的 `filters` 是模型训练良好的标志之一，未收敛或过拟合模型的卷积层 `filters` 会出现很多 noise。
- 观察 `filters` 和 `feature maps` 的图片，特别是第一层卷积的图片可以总结出该层所关注的图片特征，这有助于我们理解卷积神经网络的工作原理。

这里将 [MXNet Model Zoo](https://mxnet.incubator.apache.org/model_zoo/index.html) 中三个 CNN 模型，`Inception-BN`，`Resnet-152`，和 `VGG16` 的 `filters` 当成图像输出到 `TensorBoard`，并将这三组 `filters` 作用于一张黑天鹅的图片（来自验证数据集 `val_256_q90.rec`）上观察 `feature maps`。

可以看出三个模型的 filters 都表现出良好的光滑性和规律性，彩色 filters 负责提取原始图片前景和背景的局部特征，灰白图片负责提取图片中物体的轮廓特征。复现代码和解释请点击[这里](https://github.com/reminisce/mxboard-demo#visualizing-filters-of-convnets)。

## 可视化图片的 `embedding`

最后这个例子比较有趣。Embedding 在自然语言处理中是一个常用的概念，它是真实世界中物体在高维向量空间中的表示。我们也可以借用此概念到卷积神经网络中。卷积神经网络最后一个全连接层的输出可以看成是一个 `batch_size` 行、`num_labels` 列的矩阵，每一行作为一个 `num_labels` 维的向量就是对应输入图片的 embedding。本质上这个 embedding 就是卷积神经网络对图片的编码，softmax 层通过此编码来判断图片所属类别。当理解了图片 embedding 的概念后，我们就可以把一个图片集的所有 embedding 通过没有 softmax 层的卷积神经网络求出来，调用 MXBoard 的 `add_embedding` API，从而来观察他们在二维或者三维空间中的聚合效应，即同类别图片应该聚合在一起。
这里我们从上一个例子里的验证数据集中随机选取了$2304$张图片，用 Resnet-152 模型算出了它们的 embeddings，用 MXBoard 写入事件文件，并由 TensorBoard 读取。

这里$2304$张图片的 embeddings 默认由 PCA 算法压缩到了三维空间，不过图片聚合效应似乎不是那么明显，这是因为 PCA 算法不能保持原始物体之间的空间关系。因此，我们选用 TensorBoard 界面上提供的 `t-SNE` 算法，重新对 embeddings 进行降维操作，这是个动态的过程。

随着 `t-SNE` 算法的收敛，可以很明显地看到图片集在三维空间中被分成了几类。

最后我们来验证一下图片分类是否正确。在 TensorBoard GUI的右上角输入”`dog`”，所有打了”`dog`”标记的图片将被高亮。拖动并放大至高亮图片处，可以看到很多狗的图片，这表明预训练的 Resnet-152 模型是准确的。
全部代码和具体说明请点[这里](https://github.com/reminisce/mxboard-demo#visualizing-convnet-codes-as-embeddings)。